# Feature Dimensionality Reduction

In [ ]:
import numpy as np

# ML utils
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
train_path = "output/emnist_train_features.npz"
test_path = "output/emnist_test_features.npz"

In [ ]:
with np.load(train_path) as data:
    X_train = data['X_train']
    y_train = data['y_train']

with np.load(test_path) as data:
    X_test = data['X_test']
    y_test = data['y_test']

print(f"Train set shape: {X_train.shape}, Train labels shape: {y_train.shape}")
print(f"Test set shape: {X_test.shape}, Test labels shape: {y_test.shape}")